In [1]:
import numpy as np
import pandas as pd


In [2]:
with open('../data/BX-Books.csv') as f:
    Books = f.readlines()
    for i in range(len(Books)):
        Books[i] = Books[i].strip('"\n').split('";"')

In [3]:
Books_df = pd.DataFrame(data = Books[1:], columns= Books[0])

In [7]:
Books_df['BookID'] = range(len(Books_df))

In [12]:
Books_df.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'BookID'],
      dtype='object')

In [19]:
Books_df = Books_df.reindex(columns=['BookID', 'ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'])


In [20]:
Books_df.to_csv('../data/data_processed/Books.csv',index= False)

In [1]:
with open('../data/BX-Users.csv') as f:
    Users = f.readlines()

    

In [2]:
for i in range(len(Users)):
        Users[i] = Users[i].strip('\n').split('";')
        for j in range(len(Users[i])):
                Users[i][j] = Users[i][j].replace('"','')

In [3]:
Users

[['User-ID', 'Location', 'Age'],
 ['1', 'nyc, new york, usa', 'NULL'],
 ['2', 'stockton, california, usa', '18'],
 ['3', 'moscow, yukon territory, russia', 'NULL'],
 ['4', 'porto, v.n.gaia, portugal', '17'],
 ['5', 'farnborough, hants, united kingdom', 'NULL'],
 ['6', 'santa monica, california, usa', '61'],
 ['7', 'washington, dc, usa', 'NULL'],
 ['8', 'timmins, ontario, canada', 'NULL'],
 ['9', 'germantown, tennessee, usa', 'NULL'],
 ['10', 'albacete, wisconsin, spain', '26'],
 ['11', 'melbourne, victoria, australia', '14'],
 ['12', 'fort bragg, california, usa', 'NULL'],
 ['13', 'barcelona, barcelona, spain', '26'],
 ['14', 'mediapolis, iowa, usa', 'NULL'],
 ['15', 'calgary, alberta, canada', 'NULL'],
 ['16', 'albuquerque, new mexico, usa', 'NULL'],
 ['17', 'chesapeake, virginia, usa', 'NULL'],
 ['18', 'rio de janeiro, rio de janeiro, brazil', '25'],
 ['19', 'weston, ,', '14'],
 ['20', 'langhorne, pennsylvania, usa', '19'],
 ['21', 'ferrol / spain, alabama, spain', '46'],
 ['22', 'er

In [6]:
Users_df = pd.DataFrame(data = Users[1:], columns= Users[0])

In [7]:
Users_df.Age = Users_df.Age.replace('NULL',np.nan)

In [8]:
Users_df

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [9]:
Users_df.to_csv('../data/data_processed/Users.csv',index= False)

In [21]:
with open('../data/BX-Book-Ratings.csv') as f:
    Book_Ratings = f.readlines()
    for i in range(len(Book_Ratings)):
        Book_Ratings[i] = Book_Ratings[i].strip('"\n').split('";"')

In [22]:
Book_Ratings_df = pd.DataFrame(data = Book_Ratings[1:], columns= Book_Ratings[0])

In [23]:
Book_Ratings_df

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [71]:
Book_Ratings_df.to_csv('../data/data_processed/Book_Ratings.csv',index= False)